In [ ]:
from __future__ import print_function
import optparse    # for sorting options

# the Interface object is required by this script
import pyrosetta
import pyrosetta.rosetta as rosetta
from rosetta.protocols.scoring import Interface
from pyrosetta import init, Pose, pose_from_file, Vector1, get_fa_scorefxn, ScoreFunction, standard_packer_task, standard_task_factory, PyMOLMover
from pyrosetta.rosetta import core, protocols
from pyrosetta.teaching import fa_atr, fa_rep, fa_sol, hbond_bb_sc, hbond_sc

init()  # WARNING: option '-constant_seed' is for testing only! MAKE SURE TO REMOVE IT IN PRODUCTION RUNS!!!!!
import os; #os.chdir('.test.output')

def scanning(pdb_filename, partners, mutant_aa = 'A',
        interface_cutoff = 8.0, output = False,
        trials = 1, trial_output = ''):
    # 1. create a pose from the desired PDB file
    pose = Pose()
    pose_from_file(pose, "TLR2_6.clean.pdb")
    # 2. setup the docking FoldTree and other related parameters
    dock_jump = 1
    movable_jumps = Vector1([dock_jump])
    protocols.docking.setup_foldtree(pose, "A_B", movable_jumps)

    # 3. create ScoreFuncions for the Interface and "ddG" calculations
    # the pose's Energies objects MUST be updated for the Interface object to
    #    work normally
    scorefxn = get_fa_scorefxn() #  create_score_function('standard')
    scorefxn(pose)    # needed for proper Interface calculation

    # setup a "ddG" ScoreFunction, custom weights
    ddG_scorefxn = ScoreFunction()
    ddG_scorefxn.set_weight(core.scoring.fa_atr, 0.44)
    ddG_scorefxn.set_weight(core.scoring.fa_rep, 0.07)
    ddG_scorefxn.set_weight(core.scoring.fa_sol, 1.0)
    ddG_scorefxn.set_weight(core.scoring.hbond_bb_sc, 0.5)
    ddG_scorefxn.set_weight(core.scoring.hbond_sc, 1.0)

    # 4. create an Interface object for the pose
    interface = Interface(dock_jump)
    interface.distance(interface_cutoff)
    interface.calculate(pose)
    for trial in range( trials ):
        ddG_mutants = {}
        for i in range(1, pose.total_residue() + 1):
            # for residues at the interface
            if interface.is_interface(i) == True:
                # this way you can TURN OFF output by providing False arguments
                #    (such as '', the default)
                filename = ''
                if output:
                    filename = pose.pdb_info().name()[:-4] + '_' +\
                        pose.sequence()[i-1] +\
                        str(pose.pdb_info().number(i)) + '->' + mutant_aa
                # determine the interace score change upon mutation
                ddG_mutants[i] = interface_ddG(pose, i, mutant_aa,
                    movable_jumps, ddG_scorefxn, interface_cutoff, filename )
        print( '='*80 )
        print( 'Mutants (PDB numbered)\t\"ddG\" (interaction dependent score change)' )
        residues = list( ddG_mutants.keys() )  # list(...) conversion is for python3 compatbility
        residues.sort()    # easier to read
        display = [pose.sequence()[i - 1] +
           str(pose.pdb_info().number(i)) + mutant_aa + '\t' +
           str(ddG_mutants[i]) + '\n'
        for i in residues]
        print( ''.join(display)[:-1] )
        print( '='*80 )